<a href="https://colab.research.google.com/github/MayankkumarTank/16-bit-Microprocessor-without-interlocked-pipeline-stages-in-verilog/blob/master/NMT_preprosseing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
% cd /content/drive/MyDrive/NLP

/content/drive/MyDrive/NLP


In [ ]:
%ls

Hindi_English_Truncated_Corpus.csv  newdata.csv


In [ ]:
import pandas as pd
import numpy as np
import os
import string
from string import digits
import matplotlib.pyplot as plt
%matplotlib inline
import re

import seaborn as sns
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
from keras.layers import Input, LSTM, Embedding, Dense
from keras.models import Model

In [ ]:
data= pd.read_csv("Hindi_English_Truncated_Corpus.csv")

data.head()

,source,english_sentence,hindi_sentence
0,ted,politicians do not have permission to do what ...,"राजनीतिज्ञों के पास जो कार्य करना चाहिए, वह कर..."
1,ted,"I'd like to tell you about one such child,",मई आपको ऐसे ही एक बच्चे के बारे में बताना चाहू...
2,indic2012,This percentage is even greater than the perce...,यह प्रतिशत भारत में हिन्दुओं प्रतिशत से अधिक है।
3,ted,what we really mean is that they're bad at not...,हम ये नहीं कहना चाहते कि वो ध्यान नहीं दे पाते
4,indic2012,.The ending portion of these Vedas is called U...,इन्हीं वेदों का अंतिम भाग उपनिषद कहलाता है।


In [ ]:
data['source'].value_counts()

tides        49857
ted          38803
indic2012    36167
Name: source, dtype: int64

In [ ]:
#checking for missing values or null
pd.isnull(data).sum()

source              0
english_sentence    2
hindi_sentence      0
dtype: int64

In [ ]:
data = data[data['english_sentence'].notna()]

In [ ]:
#checking for missing values or null
pd.isnull(data).sum()

source              0
english_sentence    0
hindi_sentence      0
dtype: int64

In [ ]:
data.drop_duplicates(inplace=True)

data.shape

(124827, 3)

In [ ]:
#Convert to Lowercase 
data['english_sentence'] = data['english_sentence'].apply(lambda x: x.lower())
data['hindi_sentence'] = data['hindi_sentence'].apply(lambda x: x.lower())

In [ ]:
#Removing the Quotes
data['english_sentence'] = data['english_sentence'].apply(lambda x: re.sub("'", '', x))
data['hindi_sentence'] = data['hindi_sentence'].apply(lambda x: re.sub("'", '', x))

In [ ]:
data.head(10)

In [ ]:
#special characters
exclude = set(string.punctuation) # Set of all special characters
print(exclude)

{'(', ']', '^', '#', ':', '&', '{', '}', '[', '`', '_', "'", '>', '+', '?', '"', '~', '=', '<', '$', '/', '%', '.', '|', ')', '\\', '-', '*', ';', '@', '!', ','}


In [ ]:
# Remove all the special characters
data['english_sentence'] = data['english_sentence'].apply(lambda sen : ''.join(char for char in sen if char not in exclude))
data['hindi_sentence'] = data['hindi_sentence'].apply(lambda sen : ''.join(char for char in sen if char not in exclude))

In [ ]:
data.head(10)

In [ ]:
#Removing the numbers from all sentences/records
all_digits = str.maketrans('', '', digits)
#print(digits)

data['english_sentence'] = data['english_sentence'].apply(lambda sen : sen.translate(all_digits))
data['hindi_sentence'] = data['hindi_sentence'].apply(lambda sen : sen.translate(all_digits))

data['hindi_sentence'] = data['hindi_sentence'].apply(lambda sen : re.sub("[१२३४५६७८९०]", "", sen))

In [ ]:
#removing the extra spaces
data['english_sentence'] = data['english_sentence'].apply(lambda sen : sen.strip())
data['hindi_sentence'] = data['hindi_sentence'].apply(lambda sen : sen.strip())
data['english_sentence'] = data['english_sentence'].apply(lambda sen : re.sub(" +", " ", sen))
data['hindi_sentence'] = data['hindi_sentence'].apply(lambda sen : re.sub(" +", " ", sen))

In [ ]:
#adding start and end tokens in hindi sentence
data['hindi_sentence'] = data['hindi_sentence'].apply(lambda x : 'START_ '+ x + ' _END')

In [ ]:
data.head(6)

,source,english_sentence,hindi_sentence
0,ted,politicians do not have permission to do what ...,START_ राजनीतिज्ञों के पास जो कार्य करना चाहिए...
1,ted,id like to tell you about one such child,START_ मई आपको ऐसे ही एक बच्चे के बारे में बता...
2,indic2012,this percentage is even greater than the perce...,START_ यह प्रतिशत भारत में हिन्दुओं प्रतिशत से...
3,ted,what we really mean is that theyre bad at not ...,START_ हम ये नहीं कहना चाहते कि वो ध्यान नहीं ...
4,indic2012,the ending portion of these vedas is called up...,START_ इन्हीं वेदों का अंतिम भाग उपनिषद कहलाता...
5,tides,the then governor of kashmir resisted transfer...,START_ कश्मीर के तत्कालीन गवर्नर ने इस हस्तांत...
